In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy.signal
import scipy.special
from scipy.special import erfc
from math import log, log2, sqrt
from ipywidgets import interact, IntSlider, FloatSlider, Layout

In [2]:
# The function grayCode accepts a number n and returns an array g containing
# the Gray coding for numbers with n bits.
# This is done by recursively calling the function gray_code_recurse
def grayCode(n):
    def gray_code_recurse (g,n):
        k=len(g)
        if n<=0:
            return
        else:
            for i in range (k-1,-1,-1):
                char='1'+g[i]
                g.append(char)
            for i in range (k-1,-1,-1):
                g[i]='0'+g[i]

            gray_code_recurse (g,n-1)

    g=['0','1']
    gray_code_recurse(g,n-1)
    return g


# The function naturalBinaryCoding accepts a number n and returns an array
# with binary numbers with n bits (or equivalently up to the number 2**n, which results
# by left shifting 1 by n bits)
def naturalBinaryCoding(n):
    binary_levels = []
    for i in range(1 << n):
        binary_levels.append('{:0{}b}'.format(i, n))
    return binary_levels


# The function generateRandomBits generates n_bits random binary digits
def generateRandomBits(n_bits):
    bitstream = []
    for i in range(n_bits):
        random_bit = random.randint(0, 1)
        bitstream.append(random_bit)
    return bitstream


# The function createLevels divides the range [-A, A] into L-1 equal intervals,
# so that it always contains the endpoints of the range
def createLevels(A, L):
    y = []
    step = (A - (-A)) / (L - 1)
    for i in range(L):
        y.append(-A + i * step)
    return y


# The function createSymbols takes an array with bits as an argument and
# groups neighboring digits into symbols with length k
def createSymbols(k, bitstream):
    n_bits = len(bitstream)
    symbols = []
    for i in range(0, n_bits - k + 1, k):
        symbol = ""
        for j in range(k):
            symbol += str(bitstream[i+j])
        symbols.append(symbol)
    return symbols


# The function rootRaisedCosine creates a root raised cosine pulse
# with a roll-off coefficient and order determined by the sampling rate (nsample)
# and the delay it will introduce (delay)
def rootRaisedCosine(nsamp, roll_off, delay):
    F0 = 0.5 / nsamp
    Fd = 1
    Fs = Fd * nsamp
    Td = 1 / Fd
    Ts = 1 / Fs
    F1 = F0 * (1 - roll_off)
    F2 = F0 * (1 + roll_off)
    filter_order = 2 * nsamp * delay

    t = np.arange(0, filter_order, Td)
    h = []
    for i in range(len(t)):
        t_shifted = t[i] - filter_order / 2
        if t_shifted == 0:
            h.append(np.sqrt(2 * F0) *(1 + roll_off * ((4 / np.pi) - 1)))
        elif t_shifted == 1 / 8 / roll_off / F0 or t_shifted == - 1 / 8 / roll_off / F0 :
            h.append((roll_off * np.sqrt(F0)) * ((1 + 2 / np.pi) * np.sin(np.pi / 4 / roll_off) + (1 - 2 / np.pi) * np.cos(np.pi / 4 / roll_off)))
        else:
            factor1 = np.sqrt(2 * F0) / (1 - 64 * roll_off* roll_off * F0 * F0 * t_shifted * t_shifted)
            factor2 = np.sin(2 * np.pi * F1 * t_shifted) / (2 * np.pi * F0 * t_shifted)
            factor3 = (4 * roll_off / np.pi) * np.cos(2 * np.pi * F2 * t_shifted) 
            h.append(factor1 * (factor2 + factor3))
    
    return t,h


# The function upSample increases the number of samples of a signal signal by adding nsamp-1
# zeros after each sample of the signal
def upSample(signal, nsamp):
    upSampled = []
    for i in range(len(signal) * nsamp):
        if i % nsamp == 0:
            upSampled.append(signal[i // nsamp])
        else:
            upSampled.append(0)
    return upSampled


# The function downSample reduces the sampling frequency of a signal signal by a factor of nsamp
# by keeping only the samples that are multiples of nsamp (0, nsamp, 2*nsamp, ...)
def downSample(signal, nsamp):
    downSampled = []
    for i in range(0, len(signal), nsamp):
        downSampled.append(signal[i])
        
    return downSampled


# Adds white Gaussian noise with mean value μ (mu) and variance σ^2 (sigma)
def generateAWGN(signal, mu, sigma):
    noise = sigma * np.random.randn(len(signal)) + mu
    return noise


global filt, g_delay, g_nsamp
def interactive_signal_processing(n_bits=10000, L=8, roll_off=0.40, nsamp=20, delay=5):
    global filt, g_delay, g_nsamp
    g_delay = delay
    g_nsamp = nsamp
    bitstream = generateRandomBits(n_bits)
    A = L - 1
    k = int(log(L, 2))
    y_levels = createLevels(A, L)
    symbols = createSymbols(k, bitstream)
    gray_encoding = grayCode(k)

    x_gray = [y_levels[gray_encoding.index(symbol)] for symbol in symbols]
    t_filter, h_filter = rootRaisedCosine(nsamp, roll_off, delay)
    filt = h_filter
    y = upSample(x_gray, nsamp)
    y_transmitted = scipy.signal.convolve(y, h_filter)
    y_received = scipy.signal.convolve(y_transmitted, h_filter)
    y_final = downSample(y_received, nsamp)
    y_final = y_final[2 * delay: len(y_final) - 2 * delay]

    # Adjusted subplot configuration for three plots
    fig, axs = plt.subplots(1, 3, figsize=(20, 5))  # Adjust figsize accordingly

    # Plotting the root raised cosine filter in the first subplot
    axs[0].plot(t_filter, h_filter)
    axs[0].grid(True)
    axs[0].set_title('Root Raised Cosine Filter')
    axs[0].set_xlabel('Time')
    axs[0].set_ylabel('Amplitude')

    # Other plots remain the same, adjusting index for axs
    t = np.arange(0, len(y[:10]))
    axs[1].plot(t, y_final[:10])  # Adjusted index
    axs[1].stem(t, x_gray[:10])  # Adjusted index
    axs[1].legend(['Received', 'Transmitted'])  # Adjusted index
    axs[1].grid(True)
    axs[1].set_title('Signal Visualization')
    axs[1].set_xlabel('Time')
    axs[1].set_ylabel('Amplitude')

    # Power Spectral Density plot adjustment
    f, Pxx_den = scipy.signal.welch(y_received, window='hamming', nperseg=8192)
    Pxx_den = 10 * np.log10(Pxx_den)
    axs[2].plot(f, Pxx_den)  # Adjusted index
    axs[2].grid(True)
    axs[2].set_title('Power Spectral Density of the Received Signal')
    axs[2].set_xlabel('Normalized Frequency')
    axs[2].set_ylabel('Power/Frequency [dB]')
    plt.tight_layout()

# Widget setup
interact(interactive_signal_processing,
         n_bits=IntSlider(min=10000, max=100000, step=10000, value=10000, description='Bitstream Length', style={'description_width': 'initial'}, layout=Layout(width='100%')),
         L=IntSlider(min=2, max=16, step=1, value=8, description='ASK Levels', style={'description_width': 'initial'}, layout=Layout(width='100%')),
         roll_off=FloatSlider(min=0.1, max=1.0, step=0.1, value=0.4, description='Roll-off Factor', style={'description_width': 'initial'}, layout=Layout(width='100%')),
         nsamp=IntSlider(min=10, max=40, step=1, value=20, description='nsamp', style={'description_width': 'initial'}, layout=Layout(width='100%')),
         delay=IntSlider(min=1, max=10, step=1, value=5, description='Group Delay', style={'description_width': 'initial'}, layout=Layout(width='100%')))

interactive(children=(IntSlider(value=10000, description='Bitstream Length', layout=Layout(width='100%'), max=…

<function __main__.interactive_signal_processing(n_bits=10000, L=8, roll_off=0.4, nsamp=20, delay=5)>

In [3]:
# This function wraps the signal processing and BER calculation, making parameters adjustable via sliders
def interactive_signal_processing(n_bits=80000, L=16, roll_off=0.7, nsamp=16, delay=4):
    bitstream = generateRandomBits(n_bits)
    A = L - 1
    k = int(log2(L))
    y_levels = createLevels(A, L)
    symbols = createSymbols(k, bitstream)
    gray_encoding = grayCode(k)

    x_gray = [y_levels[gray_encoding.index(symbol)] for symbol in symbols]
    t_filter, filt = rootRaisedCosine(nsamp, roll_off, delay)
    y = upSample(x_gray, nsamp)
    y_transmitted = scipy.signal.convolve(y, filt)

    # Calculating BER theoretically and via simulation
    berTheoretical = []
    berSimulation = []
    EbN0_max = 20

    for EbN0_db in range(1, EbN0_max):
        EbN0 = 10 ** (EbN0_db / 10)
        SNR_db = EbN0_db - 10 * np.log10(nsamp / 2 / k)
        SNR = 10 ** (SNR_db / 10)
        P = sum(y_transmitted * y_transmitted) / len(y_transmitted)
        Pn = P / SNR

        noise = np.random.normal(0, sqrt(Pn), len(y_transmitted))
        y_noisy = y_transmitted + noise

        z_received = scipy.signal.convolve(y_noisy, filt)
        z_final = downSample(z_received, nsamp)
        z_final = z_final[2 * delay : len(z_final) - 2 * delay]

        errors = 0
        for i in range(len(z_final)):
            differences = np.abs(y_levels - z_final[i])
            index = np.argmin(differences)
            if y_levels[index] != x_gray[i]:
                errors += 1
        
        berSimulation.append(errors / n_bits)
        Pe = (L - 1) / L * erfc(sqrt(3 * k / (L**2 - 1) * EbN0))
        berTheoretical.append(Pe / k)
    
    # Plotting the BER
    plt.figure(figsize=(10, 6))
    plt.semilogy( berTheoretical, label='Theoretical')
    plt.semilogy(berSimulation, 'o', label='Simulation')
    plt.xlabel('Eb/N0 (dB)')
    plt.ylabel('Bit Error Rate')
    plt.title('BER Curve for L-ASK')
    plt.legend()
    plt.grid(True)
    plt.show()

# Updated function call with full-width sliders
interact(interactive_signal_processing,
         n_bits=IntSlider(min=10000, max=100000, step=10000, value=80000, description='Binary Sequence Length', layout=Layout(width='100%')),
         L=IntSlider(min=2, max=32, step=1, value=16, description='ASK Levels', layout=Layout(width='100%')),
         roll_off=FloatSlider(min=0.1, max=1.0, step=0.1, value=0.7, description='Roll-off', layout=Layout(width='100%')),
         nsamp=IntSlider(min=8, max=32, step=1, value=16, description='nsamp', layout=Layout(width='100%')),
         delay=IntSlider(min=1, max=8, step=1, value=4, description='Group Delay', layout=Layout(width='100%')))

interactive(children=(IntSlider(value=80000, description='Binary Sequence Length', layout=Layout(width='100%')…

<function __main__.interactive_signal_processing(n_bits=80000, L=16, roll_off=0.7, nsamp=16, delay=4)>